In [5]:
%matplotlib notebook
# starter code
import random
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
import networkx as nx
from graph_tool.all import load_graph, shortest_distance, GraphView, pseudo_diameter
from networkx.drawing.nx_agraph import graphviz_layout
from joblib import Parallel, delayed

from ic import simulate_cascade, observe_cascade, get_gvs, get_o2src_time, get_infection_time, gen_nontrivial_cascade
from utils import get_rank_index

seed = 123456
random.seed(seed)
np.random.seed(seed)

gtype = 'arxiv-hep-th'
g = load_graph('data/{}/graph.gt'.format(gtype))
gnx = nx.read_graphml('data/{}/graph.graphml'.format(gtype))
gnx = nx.relabel_nodes(gnx, {i: int(i) for i in gnx.nodes_iter()})
N1, N2 = 10, 100
p, q = 0.5, 0.2

# gvs = get_gvs(g, p, N2)
# pos = graphviz_layout(gnx, root=0)

In [6]:
def one_run(g, p, N1):
    mean_diffs = []
    for i in range(N1):    
        source, _, tree = simulate_cascade(g, p, return_tree=True)
        dist1 = get_infection_time(tree, source=source)
        dist2 = get_infection_time(g, source=source)
        infected_nodes = np.nonzero(dist1 != -1)[0]
        diff = np.abs((dist1 - dist2)[infected_nodes])
        mean_diffs.append(np.mean(diff))
    return np.mean(mean_diffs)
    

In [7]:
diff_mean = Parallel(n_jobs=-1)(delayed(one_run)(g, p, N1) for p in tqdm(np.linspace(0.1, 1.0, 10)))

100%|██████████| 10/10 [00:00<00:00, 33.56it/s]


In [8]:
diff_mean

[1.5078423645320196,
 1.4523513694054777,
 1.2173230314668635,
 2.1054380980337983,
 1.4428932919616237,
 1.0908995975696842,
 0.62321357823505308,
 0.45788120139453464,
 0.16274596075680495,
 0.0]